In [1]:
import pandas as pd
import os
import json
import glob
import sqlite3
import numpy as np
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Création de constantes (en majuscules par défaut) :

# Répertoire de données (data directory)
DATA_DIR = '../../data'

#Depuis l'endroit où je suis, c'est la façon d'aller au dossier data (../.. : je suis dans explo, 
# .. remonte src et hands on project)

# Répertoire temporaire (temporary directory)
TMP_DIR = '../../tmp'

##On peut remplacer cette étape par l'utilisation d'un fichier Json

In [3]:
##Lecture de fichier Json
with open('../parameters.json') as json_data:
    params = json.load(json_data)
#Params devient un dictionnaire de variables

In [4]:
#On affecte les valeurs du dictionnaire aux variables correspondantes
DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
GAS_DB = os.path.join(DATA_DIR, params['DATABASES']['GAS_DB'])

In [5]:
df_all = pd.read_pickle(os.path.join(DATA_DIR, PRICE_FILE))
df_service = pd.read_pickle(os.path.join(DATA_DIR, SERVICE_FILE))
df_station = pd.read_pickle(os.path.join(DATA_DIR, STATION_FILE))

# Calcul de l'indice de prix

In [8]:
#On convertit la date dans un meilleur format
df_all['date_releve'] = pd.to_datetime(df_all.date_releve)

In [19]:
moyenne_hebdo_df = (df_all.
                   dropna(subset=['date_releve']).
                    groupby(['id_station', 'libelle_carburant', pd.Grouper(key = 'date_releve', freq = '1W-MON')]).
                    agg({'prix' : np.mean}).
                    reset_index()
                   ).head()

In [10]:
moyenne_hebdo_df.head()

prix
id_station libelle_carburant date_releve        
1000001    E10               2015-02-09   1309.0
           Gazole            2014-01-06   1304.0
                             2014-01-13   1304.0
                             2014-01-20   1304.0
                             2014-01-27   1304.0

In [26]:
model_df = (df_all
.dropna(subset=['date_releve'])
.groupby(['id_station','libelle_carburant', pd.Grouper(key = 'date_releve', freq = '1W-MON')])
.agg({'prix' : np.mean})
.reset_index()
.merge(moyenne_hebdo_df, how='inner', on=['libelle_carburant', 'date_releve'], suffixes=('','_moyen'))
.assign(indice = lambda _df : _df['prix']/ _df['prix_moyen']
       ))

In [27]:
model_df.shape

(26481, 7)

In [28]:
model_df.head()

,id_station,libelle_carburant,date_releve,prix,id_station_moyen,prix_moyen,indice
0,1000001,E10,2015-02-09,1309.000000,1000001,1309.0,1.000000
1,1000007,E10,2015-02-09,1269.800000,1000001,1309.0,0.970053
2,1000008,E10,2015-02-09,1272.333333,1000001,1309.0,0.971989
3,1000009,E10,2015-02-09,1267.500000,1000001,1309.0,0.968296
4,1100001,E10,2015-02-09,1274.000000,1000001,1309.0,0.973262


In [29]:
model_df.indice.describe()

count    26481.000000
mean         1.023507
std          0.039994
min          0.870399
25%          0.998466
50%          1.011503
75%          1.042271
max          1.379601
Name: indice, dtype: float64